In [1]:
import requests
from bs4 import BeautifulSoup 
import json 
import scraper
from tqdm import tqdm 
import pandas as pd 
import pickle 

headers = {
    'accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53',
    'Accept-Language': 'en-US,en;q=0.9,it;q=0.8,es;q=0.7',
    'referer': 'https://www.google.com/',
}


cookies = json.load(open('cookies.json' , 'rb'))

In [2]:
url = 'https://cooking.nytimes.com/search?page='
base_url = 'https://cooking.nytimes.com/'
df = pd.DataFrame()
exceptions = 0


In [4]:
for i in (pbar:=tqdm(range(1,50))):
    try:
        page = requests.get(url+str(i) , cookies=cookies , headers=headers)
        soup = BeautifulSoup(page.text)
        data = soup.find_all('script' , attrs={'type':'application/json'})
        
        data = json.loads(data[0].contents[0])

        cards = data['props']['pageProps']['results']
        for card in cards: 
            if card['type'] == 'collection':
                df = df.append(scraper.scrape_blog(base_url+card['url']), ignore_index=True)
                pbar.set_description(f'Total recipes scraped {len(df)}, number of exceptions {exceptions}')

                
            elif card['type'] == 'recipe':
                df = df.append(json.loads(scraper.scrape_recipe(base_url+card['url'])), ignore_index=True)
                pbar.set_description(f'Total recipes scraped {len(df)}, number of exceptions {exceptions}')

        
    except Exception as e:
        exceptions += 1 
        print(e)
    finally: 
        pbar.set_description(f'Total recipes scraped {len(df)}, number of exceptions {exceptions}')


    

Total recipes scraped 0, number of exceptions 8:   2%|▏         | 1/49 [00:03<03:08,  3.92s/it]

the JSON object must be str, bytes or bytearray, not dict


Total recipes scraped 1, number of exceptions 9:   4%|▍         | 2/49 [00:10<04:27,  5.70s/it]

the JSON object must be str, bytes or bytearray, not dict


Total recipes scraped 1, number of exceptions 9:   4%|▍         | 2/49 [00:13<05:25,  6.92s/it]


KeyboardInterrupt: 

In [4]:
df


,@context,@type,name,description,author,image,totalTime,recipeYield,recipeCuisine,recipeCategory,keywords,aggregateRating,nutrition,recipeIngredient,recipeInstructions,isAccessibleForFree,hasPart,publisher,video
0,http://schema.org,Recipe,Grilled Tofu Tacos,"Inspired by zesty <a href=""https://cooking.nyt...","{'@type': 'Person', 'name': 'Kay Chun'}",[https://static01.nyt.com/images/2023/06/22/mu...,PT45M,8 tacos,,"dinner, easy, lunch, quick, weeknight, tacos, ...","chipotles in adobo, cilantro, firm tofu, sour ...","{'@type': 'AggregateRating', 'ratingValue': 3,...","{'@context': 'http://schema.org', '@type': 'Nu...","[2 tablespoons neutral oil, plus more for oili...","[{'@context': 'http://schema.org', '@type': 'H...",False,"{'@type': 'WebPageElement', 'isAccessibleForFr...","{'@context': 'http://schema.org', '@type': 'Or...",NaN
1,http://schema.org,Recipe,Classic Lemon Tart,"This classic lemon tart has a buttery, shortbr...","{'@type': 'Person', 'name': 'Melissa Clark'}",[https://static01.nyt.com/images/2023/06/07/mu...,PT4H,8 servings,french,dessert,"brown butter, egg yolk, lemon, lemon juice, le...","{'@type': 'AggregateRating', 'ratingValue': 4,...","{'@context': 'http://schema.org', '@type': 'Nu...","[8 tablespoons/113 grams unsalted butter, 1 1/...","[{'@context': 'http://schema.org', '@type': 'H...",False,"{'@type': 'WebPageElement', 'isAccessibleForFr...","{'@context': 'http://schema.org', '@type': 'Or...",NaN
2,http://schema.org,Recipe,Japanese Potato Salad With Mentaiko,A classic side dish for homemade lunch boxes o...,"{'@type': 'Person', 'name': 'J. Kenji López-Alt'}",[https://static01.nyt.com/images/2023/06/21/mu...,PT45M,4 to 6 servings,japanese,"dinner, lunch, side dish","carrot, corn, cucumber, egg, lemon juice, lemo...","{'@type': 'AggregateRating', 'ratingValue': 3,...",NaN,[1 pound russet potatoes (about 2 whole large ...,"[{'@context': 'http://schema.org', '@type': 'H...",False,"{'@type': 'WebPageElement', 'isAccessibleForFr...","{'@context': 'http://schema.org', '@type': 'Or...",NaN
3,http://schema.org,Recipe,Saimin,This beloved noodle soup — unique to Hawaii an...,"{'@type': 'Person', 'name': 'Elyse Inamine'}",[https://static01.nyt.com/images/2023/06/21/mu...,PT3H30M,10 servings,,"dinner, lunch, project, main course","dashi, egg, kombu, pork, ramen noodle, scallio...","{'@type': 'AggregateRating', 'ratingValue': 3,...",NaN,"[2 bunches scallions (8 ounces), trimmed, 1 me...","[{'@context': 'http://schema.org', '@type': 'H...",False,"{'@type': 'WebPageElement', 'isAccessibleForFr...","{'@context': 'http://schema.org', '@type': 'Or...",NaN
4,http://schema.org,Recipe,Sago Pudding,"In Southeast Asia, sago or tapioca pearls are ...","{'@type': 'Person', 'name': 'Hetty McKinnon'}",[https://static01.nyt.com/images/2023/06/16/mu...,PT2H40M,6 servings,,"easy, snack, custards and puddings, dessert","coconut cream, make-ahead, plan-ahead, southea...","{'@type': 'AggregateRating', 'ratingValue': 3,...","{'@context': 'http://schema.org', '@type': 'Nu...",[1 cup sago or small pearl tapioca (not quick-...,"[{'@context': 'http://schema.org', '@type': 'H...",False,"{'@type': 'WebPageElement', 'isAccessibleForFr...","{'@context': 'http://schema.org', '@type': 'Or...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,http://schema.org,Recipe,Panzanella With Mozzarella and Herbs,"At the height of tomato season, for every perf...","{'@type': 'Person', 'name': 'Melissa Clark'}",[https://static01.nyt.com/images/2015/07/08/di...,PT45M,6 servings,italian,"dinner, lunch, salads and dressings, appetizer...","bread, mozzarella, tomato","{'@type': 'AggregateRating', 'ratingValue': 5,...","{'@context': 'http://schema.org', '@type': 'Nu...","[4 ounces ciabatta or baguette, preferably sta...","[{'@context': 'http://schema.org', '@type': 'H...",False,"{'@type': 'WebPageElement', 'isAccessibleForFr...","{'@context': 'http://schema.org', '@type': 'Or...",NaN
30,http://schema.org,Recipe,Yotam Ottolenghi

In [ ]:
print(data['props']['pageProps']['results'][0])